# Cloud Drive Migration using Colab
This will help you migrate your files from Baidu Yun to Google Drive using Colab.

## Authorization Baidu Yun
Running the code below would install bypy on this temporary google colab instance, and generate a link for you to authorize bypy to access your Baidu Yun Data, but only under the folder "我的网盘/我的应用数据/bypy"

this will also generate the folder "我的网盘/我的应用数据/bypy" if it doesn't exist.

**Click the generated link, log in your Baidu Yun, then copy the authorization code to the input box under the generated link, press Enter.**


In [ ]:
!pip install bypy==1.6.10 # install bypy
import bypy
bp = bypy.bypy.ByPy(processes=1) # You can specify multi-process by modify processes, but currently, recommend using only 1 process. 
# multi processes may have risk of Baidu's action towards your baidu Yun. I'm not sure about that

## Prepare Files in Baidu Yun
As mentioned above, bypy is only allowed to access "我的网盘/我的应用数据/bypy", therefore, you need to

**Move/Copy the files you want to migrate into folder "我的网盘/我的应用数据/bypy"**  
**This step should be completed on your Baidu Yun Web/PC/mobile client, not here**

If you have successfully moved your files, running the code below should displays your files under "我的网盘/我的应用数据/bypy".

In [ ]:
bp.list()

## Mount Google Drive
Running the code below would mount your Google Drive to this temporary google colab instance, on '/content/drive/MyDrive'.

**Click the generated link, as you've already logged in your Google account, just copy the authorization code to the input box under the generated link, press Enter.**

In [ ]:
from google.colab import drive
gdrive_mount_path = '/content/drive'
drive.mount(gdrive_mount_path)

## Migrate(Copy) your files from Baidu Yun to Google Drive

**Specify the original folder in Baidu Yun you want to migrate from, and the destination Google Drive folder**

**Folder Only!!! not file**

In [ ]:
baidudir="" #If left blank, it means the whole "我的网盘/我的应用数据/bypy" folder
gdrivedir="baiduyun" # I don't recommend leave it blank

Actually, bypy already provides sync function, but Baidu Yun provides wrong md5 hash value (as bypy author says).  
Thus, the code below would "manually" compare the existing paths, filenames between the origin and destination.  
Because the md5 is broken, we can't really sync the files, but the code can be re-run, which is important because the colab would automatically terminate in about 6 hours(or 12 hours, not sure) 

**Run the code below, re-run it if the colab terminate before the migrate complete**

(It doesn't matter if it says "FAILED", that's just because of md5 matter)

In [ ]:
import os

full_gdrivedir = os.path.join(gdrive_mount_path, "MyDrive/", gdrivedir)
print("The origin path in Baidu Yun:", bypy.util.get_pcs_path(baidudir))
print("The destination path:", full_gdrivedir)

bp.compare(baidudir,full_gdrivedir,skip_remote_only_dirs=True)
for item in bp.result['remote']:
  if item[0] == 'D':
    bpath = os.path.join(baidudir,item[1])
    gpath = os.path.join(full_gdrivedir,item[1])
    print(bpath,gpath)
    bp.downdir(remotepath=bpath, localpath=gpath)
  elif item[0] == 'F':
    bfile = os.path.join(baidudir,item[1])
    gfile = os.path.join(full_gdrivedir,item[1])
    print(gfile)
    gpath = os.path.split(gfile)[0]
    print(bfile,gpath)
    bp.downfile(remotefile=bfile,localpath=gpath)